
# Exploring the Relationship between Air Quality and COVID-19 Infections
For this project, our goal is to investigate a **possible** relationship between Air Quality and COVID-19 Infection Rates. To that end, we merged an Air Quality dataset with a COVID-19 dataset, performed preprocessing before and after the merge, trained a regression model on the merged dataset, and performed predictions to test the accuracy of our model.

As an exploration, we attempted to exhaust all relevant regression models whose parameters have been appropriately tuned (if possible), from Support Vector Machines to Linear Regressors (we settled with the latter). All relevant preprocessing steps, from data cleaning, feature renaming and imputation, to pruning, feature selection and standardization, have also been taken.

To simplify our scope, we have focused only on California Air Quality and COVID-19 datasets, putting a geographical restriction on our approach. The time-series data were sourced directly from the United States Environmental Protection Agency (EPA) and the California State Government respectively. This choice of going abroad is motivated by the difficulty of procuring day-to-day Air Quality and COVID-19 data from the Philippines. We require day-to-day data for a more granular training set.

Because our preprocessed dataset only holds continuous quantitative values, regression is the most appropriate modeling approach.

**Important:** Because this is an exploratory project, we are not absolutely certain that there exists a significant relationship between Air Quality and COVID-19 datasets. Furthermore, our results, significant or not, can only lend support to hypotheses surrounding Air Quality and COVID-19.

## Importing and Assessing the Datasets
Let us look at the samples, features, and other details regarding the dataset so that we better know what we're dealing with. But first, we need to import the datasets into manipulable dataframes. Note: We've already checked the datasets prior to downloading them, but we include this section for completeness.

**Terminology:**
- **Dataset A** - the dataset containing day-to-day California Air Quality data based on raw pollutant measures, from 2020-2022.
Raw source: [https://www.epa.gov/outdoor-air-quality-data/download-daily-data](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)
- **Dataset B** - the dataset containing day-to-day New York COVID-19 cases data, from 2020 to 2022.
Raw source: [https://data.ca.gov/dataset/covid-19-time-series-metrics-by-county-and-state](https://data.ca.gov/dataset/covid-19-time-series-metrics-by-county-and-state)

Note that the reason we started with dataset fragments for Dataset A is because downloading the datasets by year, by state, and then by pollutant type is the only way to do so in the EPA website.

### Importing Dataset A
In this section, we will import the pollutant-by-pollutant and then year-by-year fragments (after accumulation) of Dataset A, namely dataA2020, dataA2021, and dataA2022.
Next, we'll make preliminary observations on them, and then concatenate them together to form our desired dataset, dataA, that goes from the start of the pandemic on 2020 until the end of 2022.

In [1537]:
# Set Jupyter Notebook to clear all workspace variables before the next complete run
%reset -f
# Set Jupyter Notebook to print out all interactions.
# This enables displaying the contents of a variable or function return value from any part of the code and
# not just at the end of code block.
%config InteractiveShell.ast_node_interactivity = 'all'

# Fetching all Air Quality datasets into their dataframes
# Perform immediate concatenation per year
import os
import pandas as pd

# Initialize accumulator datasets using CO (carbon dioxide) datasets (these are temporary)
# NOTE: Dataset fragments are all in './CAdata/'
datadir = 'CAdata/'
colidx = [0,2,4,17]                             # column indexes to use (already looked at CA datasets)
innerkeys = ['Date', 'Site ID', 'COUNTY']       # Merge on Date, Site ID, and COUNTY

# Because date is an important factor to us, as we read the CSV files, we immediately parse their dates
# Note that colidx is applied here, which cherry picks the columns to use from the datasets
# wherein the columns contain extraneous information of coordinates and site name.
dataA2020 = pd.read_csv(datadir + 'ca-2020-co.csv', parse_dates=True, usecols=colidx)
dataA2021 = pd.read_csv(datadir + 'ca-2021-co.csv', parse_dates=True, usecols=colidx)
dataA2022 = pd.read_csv(datadir + 'ca-2022-co.csv', parse_dates=True, usecols=colidx)

# Iterate through the CAdata directory using os.scandir, and then get all the datasets there
# Merge the remaining datasets other than ca-year-co to dataAyear
with os.scandir(datadir) as datasets:
    for dataset in datasets:
        if dataset.is_file() and 'co' not in dataset.name and 'ca' in dataset.name:
            temp = pd.read_csv(dataset, parse_dates=True, usecols=colidx)
            if '2020' in dataset.name:
                dataA2020 = pd.merge(dataA2020, temp, how='outer', on=innerkeys)
            elif '2021' in dataset.name:
                dataA2021 = pd.merge(dataA2021, temp, how='outer', on=innerkeys)
            elif '2022' in dataset.name:
                dataA2022 = pd.merge(dataA2022, temp, how='outer', on=innerkeys)

We used outer join for the merge in order to increase the number of records that we have, especially because the volume of records is crucial for training regressors. We'll handle the NaN values later on with imputation.

Now let's take a peek at the dataset A year-by-year fragments. As expected, use of outer join has left us with a lot of NaN values, but thankfully, those values are properly surrounded by existing values on a time-series, hence imputation would be appropriately applied to this. We also find several columns that we could still drop.

In [1538]:
# At this point, dataA2020, dataA2021, and dataA2022 have accumulated the CA dataset year fragments.
dataA2020.head()
dataA2021.head()
dataA2022.head()

,Date,Site ID,Daily Max 8-hour CO Concentration,COUNTY,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
0,01/01/2020,60010009,0.7,Alameda,21.3,0.020,NaN,NaN,13.1,NaN
1,01/02/2020,60010009,0.7,Alameda,29.4,0.011,NaN,NaN,7.6,NaN
2,01/03/2020,60010009,0.9,Alameda,31.9,0.007,NaN,NaN,15.0,NaN
3,01/04/2020,60010009,0.8,Alameda,23.0,0.024,NaN,NaN,9.5,NaN
4,01/05/2020,60010009,0.7,Alameda,22.9,0.030,NaN,NaN,6.1,NaN


,Date,Site ID,Daily Max 8-hour CO Concentration,COUNTY,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
0,01/01/2021,60010009,0.7,Alameda,25.0,0.019,NaN,NaN,14.2,NaN
1,01/02/2021,60010009,0.7,Alameda,25.2,0.014,NaN,NaN,9.4,NaN
2,01/03/2021,60010009,0.7,Alameda,21.9,0.016,NaN,NaN,13.7,NaN
3,01/04/2021,60010009,0.7,Alameda,22.2,0.030,NaN,NaN,8.2,NaN
4,01/05/2021,60010009,0.6,Alameda,19.1,0.023,NaN,NaN,6.6,NaN


,Date,Site ID,Daily Max 8-hour CO Concentration,COUNTY,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
0,01/01/2022,60010009,0.7,Alameda,22.9,0.028,NaN,NaN,13.2,NaN
1,01/02/2022,60010009,0.6,Alameda,25.5,0.021,NaN,NaN,8.2,NaN
2,01/03/2022,60010009,0.6,Alameda,20.1,0.029,NaN,NaN,3.8,NaN
3,01/04/2022,60010009,0.4,Alameda,23.1,0.025,NaN,NaN,3.1,NaN
4,01/05/2022,60010009,0.4,Alameda,29.2,0.023,NaN,NaN,4.7,NaN


The initial datasets drops the 'Site ID' Column and then groups them by year. These datasets contain the air contents as well as the date in which they are recorded. They are grouped depending on their year since the initial datasets are limited to their own year.

In [1539]:
# We then parse Dates to date
dataA2020['Date'] = pd.to_datetime(dataA2020['Date'])
dataA2021['Date'] = pd.to_datetime(dataA2021['Date'])
dataA2022['Date'] = pd.to_datetime(dataA2022['Date'])
dataA2020.head()
dataA2021.head()
dataA2022.head()

# Group data by Date and Site ID, then by Date again to remove the Site ID feature
# Result would be mean measurements per day
dataA2020 = dataA2020.drop(columns=['Site ID'], axis=1).groupby(by=['Date']).mean()
dataA2021 = dataA2021.drop(columns=['Site ID'], axis=1).groupby(by=['Date']).mean()
dataA2022 = dataA2022.drop(columns=['Site ID'], axis=1).groupby(by=['Date']).mean()
dataA2020.head()
dataA2021.head()
dataA2022.head()

,Date,Site ID,Daily Max 8-hour CO Concentration,COUNTY,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
0,2020-01-01,60010009,0.7,Alameda,21.3,0.020,NaN,NaN,13.1,NaN
1,2020-01-02,60010009,0.7,Alameda,29.4,0.011,NaN,NaN,7.6,NaN
2,2020-01-03,60010009,0.9,Alameda,31.9,0.007,NaN,NaN,15.0,NaN
3,2020-01-04,60010009,0.8,Alameda,23.0,0.024,NaN,NaN,9.5,NaN
4,2020-01-05,60010009,0.7,Alameda,22.9,0.030,NaN,NaN,6.1,NaN


,Date,Site ID,Daily Max 8-hour CO Concentration,COUNTY,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
0,2021-01-01,60010009,0.7,Alameda,25.0,0.019,NaN,NaN,14.2,NaN
1,2021-01-02,60010009,0.7,Alameda,25.2,0.014,NaN,NaN,9.4,NaN
2,2021-01-03,60010009,0.7,Alameda,21.9,0.016,NaN,NaN,13.7,NaN
3,2021-01-04,60010009,0.7,Alameda,22.2,0.030,NaN,NaN,8.2,NaN
4,2021-01-05,60010009,0.6,Alameda,19.1,0.023,NaN,NaN,6.6,NaN


,Date,Site ID,Daily Max 8-hour CO Concentration,COUNTY,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
0,2022-01-01,60010009,0.7,Alameda,22.9,0.028,NaN,NaN,13.2,NaN
1,2022-01-02,60010009,0.6,Alameda,25.5,0.021,NaN,NaN,8.2,NaN
2,2022-01-03,60010009,0.6,Alameda,20.1,0.029,NaN,NaN,3.8,NaN
3,2022-01-04,60010009,0.4,Alameda,23.1,0.025,NaN,NaN,3.1,NaN
4,2022-01-05,60010009,0.4,Alameda,29.2,0.023,NaN,NaN,4.7,NaN


,Daily Max 8-hour CO Concentration,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
Date,,,,,,,
2020-01-01,0.636559,21.474615,0.030969,0.008000,20.491525,15.203015,0.972549
2020-01-02,0.697647,26.564463,0.028227,0.013000,18.468085,12.619178,1.077083
2020-01-03,0.845238,31.382500,0.026872,0.018990,21.129032,14.427778,1.044681
2020-01-04,1.049138,32.154152,0.029897,0.011074,29.707692,16.439510,0.983929
2020-01-05,0.792941,22.521138,0.032683,0.009000,13.905263,10.883221,0.766667


,Daily Max 8-hour CO Concentration,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
Date,,,,,,,
2021-01-01,0.665000,22.645000,0.030844,0.021500,16.342105,14.104615,0.950000
2021-01-02,0.657895,22.458475,0.028398,0.010000,18.055556,12.370199,0.820455
2021-01-03,0.663158,21.499145,0.029293,0.008000,19.090909,14.147368,0.720930
2021-01-04,0.709740,27.123196,0.029920,0.006776,24.549550,14.059341,0.725000
2021-01-05,0.711842,27.218966,0.026878,0.027000,17.977528,12.394079,0.892857


,Daily Max 8-hour CO Concentration,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
Date,,,,,,,
2022-01-01,0.556000,21.297581,0.034212,NaN,14.615385,10.865663,0.541860
2022-01-02,0.552688,25.704167,0.032208,NaN,13.600000,8.823744,0.730508
2022-01-03,0.700000,28.399194,0.030789,NaN,18.560345,8.744785,0.925581
2022-01-04,0.762667,29.063934,0.029168,NaN,21.894737,9.966061,1.142857
2022-01-05,1.031157,41.096725,0.022163,0.01166,35.697400,16.667782,1.211684


## Merging of Temporary Datasets into one dataset (Dataset A)
This is where Air Particles concentration from each year are merged into one whole dataset (2020-2023) which will be used later on.

In [1540]:
dataA = pd.concat([dataA2020, dataA2021, dataA2022])    # Combine the three datasets
print("Combined dataset (Date set as index)")
dataA

Combined dataset (Date set as index)


,Daily Max 8-hour CO Concentration,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
Date,,,,,,,
2020-01-01,0.636559,21.474615,0.030969,0.008000,20.491525,15.203015,0.972549
2020-01-02,0.697647,26.564463,0.028227,0.013000,18.468085,12.619178,1.077083
2020-01-03,0.845238,31.382500,0.026872,0.018990,21.129032,14.427778,1.044681
2020-01-04,1.049138,32.154152,0.029897,0.011074,29.707692,16.439510,0.983929
2020-01-05,0.792941,22.521138,0.032683,0.009000,13.905263,10.883221,0.766667
...,...,...,...,...,...,...,...
2022-12-27,0.814925,25.993333,0.027554,NaN,20.254386,8.294737,0.761111
2022-12-28,0.409756,20.100000,0.030041,NaN,12.185185,7.471809,0.555769
2022-12-29,0.392063,17.505660,0.028821,0.004000,12.609091,6.344000,0.361765


## Column Renaming:
This is where we shorten the name of each air contaminants to a corresponding concise name along with their corresponsing units.

In [1541]:
new_names = ['CO conc (ppm)', 'NO2 conc (ppb)', 'O3 conc (ppm)',
             'Pb conc (ug/m3 SC)', 'PM10 conc (ug/m3 SC)',
             'PM2.5 conc (ug/m3 LC)', 'SO2 conc (ppb)']

# Rename columns
for i in range(len(new_names)):
    dataA.rename(columns={dataA.columns[i]: new_names[i]}, inplace=True)

print("Columns renamed:")
dataA.head()

Columns renamed:


,CO conc (ppm),NO2 conc (ppb),O3 conc (ppm),Pb conc (ug/m3 SC),PM10 conc (ug/m3 SC),PM2.5 conc (ug/m3 LC),SO2 conc (ppb)
Date,,,,,,,
2020-01-01,0.636559,21.474615,0.030969,0.008000,20.491525,15.203015,0.972549
2020-01-02,0.697647,26.564463,0.028227,0.013000,18.468085,12.619178,1.077083
2020-01-03,0.845238,31.382500,0.026872,0.018990,21.129032,14.427778,1.044681
2020-01-04,1.049138,32.154152,0.029897,0.011074,29.707692,16.439510,0.983929
2020-01-05,0.792941,22.521138,0.032683,0.009000,13.905263,10.883221,0.766667


### Checking null values in the dataset
This is where we checked the null values on the dataset and decide what actions are needed to be done. In this case, imputing is the best course of action that we decided to do.

In [1542]:
import seaborn as sns

# Deciding whether to drop or impute null values, so we check how many null values there are.
# Dataset A impute
print("A: Number of entries with null values:", dataA.isna().any(axis=1).sum())
print("A: Number of entries:", dataA.shape[0])

A: Number of entries with null values: 359
A: Number of entries: 1096


### Imputation
This is where we begin using an imputer to fill in the null values that the original dataset contains.

In [1543]:
# These imports are important, imputer relies on them.

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer   # Important!
from sklearn.impute import IterativeImputer     # default imputer is BayesianRidge

from sklearn.linear_model import BayesianRidge

# Other estimators (estimator = func()) to try
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

# Initialize imputer
imp = IterativeImputer(max_iter=100, random_state=1, verbose=True, estimator=BayesianRidge())
dataA[:] = imp.fit_transform(dataA)

print("After imputation:")
dataA.head()
print("A: Number of entries with null values after impute:", dataA.isna().any(axis=1).sum())
print("A: Number of entries:", dataA.shape[0])

[IterativeImputer] Completing matrix with shape (1096, 7)
[IterativeImputer] Change: 0.015027354506753809, scaled tolerance: 0.22167032967032968 
[IterativeImputer] Early stopping criterion reached.
After imputation:


,CO conc (ppm),NO2 conc (ppb),O3 conc (ppm),Pb conc (ug/m3 SC),PM10 conc (ug/m3 SC),PM2.5 conc (ug/m3 LC),SO2 conc (ppb)
Date,,,,,,,
2020-01-01,0.636559,21.474615,0.030969,0.008000,20.491525,15.203015,0.972549
2020-01-02,0.697647,26.564463,0.028227,0.013000,18.468085,12.619178,1.077083
2020-01-03,0.845238,31.382500,0.026872,0.018990,21.129032,14.427778,1.044681
2020-01-04,1.049138,32.154152,0.029897,0.011074,29.707692,16.439510,0.983929
2020-01-05,0.792941,22.521138,0.032683,0.009000,13.905263,10.883221,0.766667


A: Number of entries with null values after impute: 0
A: Number of entries: 1096


## Dataset B
This is the second dataset that we are going to use alongside dataset A. This contains the COVID-19 data, specifically the date when it was recorded, the state that it came from, the total covid cases, confirmed covid cases, probable covid cases and the sum of the new cases that arose that day.

In this dataset, we limit our data to the state of California (CA). Then, we used the date when the data was gathered as the index of the dataset. After that, we filtered dataset A with the date available in dataset B.

Finally, we checked the null values of dataset B to determine whether we need to impute or not. In this case, we don't need to impute since it has clean data.

In [1544]:
colidx = [0,1,2,3,5,6]     # column indexes to use (based on preemptively looking at dataB)
dataB = pd.read_csv('datasets/us_covid_cases_and_deaths_by_state.csv', parse_dates=True, usecols=colidx)
sum_new_cases = dataB['new_case'] + dataB['pnew_case']
dataB.drop(['new_case', 'pnew_case'], axis=1, inplace=True)
dataB['sum_new_cases'] = sum_new_cases
dataB.head()

dataB.columns = ['Date', 'State', 'Total Cases', 'Confirmed Cases', 'Probable Cases', 'Sum New Cases']

# Filter dataset B
dataB = dataB[dataB['State'] == 'CA']
dataB.head()

# Use Date as index, also drop the State
print("Dateset B finalized")
dataB['Date'] = pd.to_datetime(dataB['Date'])
dataB.set_index('Date', inplace=True)
dataB.sort_index(inplace=True)
dataB.drop('State', axis=1, inplace=True)
dataB.info()
dataB.head()

# Filter dataA with temporal restriction given by dataB
dataA = dataA[(dataA.index >= dataB.index.min()) &
              (dataA.index <= dataB.index.max())]

print("Filtered Dataset A")
dataA.head()

# With the printouts below, we find that there's no need to impute.
print("B: Number of entries with null values:", dataB.isna().any(axis=1).sum())
print("B: Number of entries:", dataB.shape[0])

,submission_date,state,tot_cases,conf_cases,sum_new_cases
0,03/11/2021,KS,297229,241035.0,0.0
1,12/01/2021,ND,163565,135705.0,809.0
2,01/02/2022,AS,11,NaN,0.0
3,11/22/2021,AL,841461,620483.0,1060.0
4,05/30/2022,AK,251425,NaN,0.0


ValueError: Length mismatch: Expected axis has 5 elements, new values have 6 elements

## Merging of Main Datasets (data = dataA + dataB)
This is where we merged the air quality dataset and the COVID-19 dataset into one dataframe called 'data'.

In [ ]:
# Merging the two datasets (dataA & dataB)
data = dataA.merge(dataB, left_index=True, right_index=True)
data

## Pruning
This is where we pruned each column of the dataset in order to reach maximum efficiency. We also removed some columns that we determined to be skewing the prediction in the wrong way.

This also shows a scatter plot of each feature matrix column (except for Total Cases) with the target matrix (Sum New Cases).

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# GENERATE SCATTERPLOTS FOR EDUCATED GUESSES OF WHAT COLUMNS TO USE
# DYNAMIC PRUNING
# IDEA: We focus on ambient level of pollutants and ignore sudden spikes in COVID-19 case data (i.e. data dumps).
# We also remove Dates where there are no new infections as their volume skews the data alot.
data = data[(data['Sum New Cases'] <= 73000) & (data['Sum New Cases'] > 1)]
data = data[(data['Total Cases'] > 0)]
data = data[(data['Confirmed Cases'] > 0)]
data = data[data['CO conc (ppm)'] <= 1.5]
# data = data[data['NO2 conc (ppb)'] > 1]
data = data[data['O3 conc (ppm)'] > 0.035]
data = data[data['Pb conc (ug/m3 SC)'] < 0.5]
data = data[data['PM10 conc (ug/m3 SC)'] < 100]
data = data[data['PM2.5 conc (ug/m3 LC)'] < 15]
data = data[data['SO2 conc (ppb)'] < 2.0]

# DROP COLUMNS HERE
data.drop(columns=['PM10 conc (ug/m3 SC)'], axis=1, inplace=True)
# data = data[['PM10 conc (ug/m3 SC)', 'Sum New Cases']]

for label in data.columns:
    if label in ['Sum New Cases', 'Total Cases', 'Confirmed Cases']: continue
    sns.set_style('dark')

    sns.relplot(x=label, y='Sum New Cases', data=data, height=3.8, aspect=1.8, kind='scatter')

print("Number of entries remaining after pruning:", data.shape[0])

### Regression Plot
This contains regression plots of each column in the feature matrix (except for Total Cases) with the target matrix (Sum New Cases).

In [ ]:
fig, axs = plt.subplots(nrows=len(data.columns), ncols=1, figsize=(16,64))

i = 0
for label in data.columns:
    if label in ["Sum New Cases", "Total Cases", "Confirmed Cases"]: continue
    sns.regplot(x=label, y='Sum New Cases', data=data, ci=95, scatter_kws={'s':100, 'facecolor':'red'}, ax=axs[i])
    i += 1

## Linear Regression
Here we initially started with the initialization of the StandardScaler. Then, we grouped the component of the dataset depending on their classification, whether they are the Target Vector (Sum New Cases) or the Feature Matrix (remaining columns in the dataframe).

Now that we have our x and y, we utilized train_test_split to separate the train data from the test data. Then finally, we started our Linear Regression with the variable 'regressor'.

In [ ]:
# Linear Regression
# PREPARE FEATURES AND TARGET DATA (normalize first)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
scaler = StandardScaler()
data
X = data.iloc[:,0:-1] # feature matrix
X = scaler.fit_transform(X)
y = data.iloc[:,-1] # target vector

# PREPARE TRAIN AND TEST DATA
# NOTE: random_state = 1
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
import sklearn.linear_model as sklm
regressor = LinearRegression()
regressor.fit(X_train, y_train)

print("Coefficients:", regressor.coef_)
print("Intercept:", regressor.intercept_)

### Predictions
This is where we use linear regression to predict values given the test data.

In [ ]:
y_pred = regressor.predict(X_test)
y_pred

## Comparison of Actual Data with Predicted Data
This is just a side by side comparison of the values of the Actual Data compared with the Predicted Data.

In [ ]:
comparison_df = pd.DataFrame({"Actual":y_test,"Predicted":y_pred})
comparison_df

## Residuals
This contains the difference between the actual value of the data with the predicted data value.

In [ ]:
residuals = y_test - y_pred
residuals

### Plotting of Actual Data with Predicted Data
This is a simple scatter plot of the Actual Data with the Predicted Data.

In [ ]:
sns.scatterplot(x=y_test, y = y_pred, s=140)
plt.xlabel('y_test data')
plt.ylabel('Predictions')

## Metrics Log
This contains the metric values output of the model when used on the merged dataset.

In [ ]:
f = open("log.out", "a")

from sklearn.metrics import mean_absolute_error
print('MAE:', mean_absolute_error(y_test,y_pred))
f.write('MAE: {}\n'.format(mean_absolute_error(y_test,y_pred)))

from sklearn.metrics import mean_squared_error
print("MSE",mean_squared_error(y_test,y_pred))
f.write('MSE: {}\n'.format(mean_squared_error(y_test,y_pred)))

import numpy as np
print("RMSE",np.sqrt(mean_squared_error(y_test,y_pred)))
f.write('RMSE: {}\n'.format(np.sqrt(mean_squared_error(y_test,y_pred))))

from sklearn.metrics import r2_score
r2 = r2_score(y_test,y_pred)
print("R2 Score:", r2)      # We use this as our main metric
f.write('R2 Score: {}\n'.format(r2))

f.write('---\n')

In [ ]:
data = data.reset_index()
NewData = data
NewData

In [ ]:
# For CO conc (ppm) & Sum New Cases
## Perform regression modeling
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

import statsmodels.api as sm
import calendar

# Convert datetime to int
#x = data['Date'].values.astype(int) / 10**9  # Convert to seconds (UNIX epoch start)
#x = x.reshape(-1, 1)


x = NewData['CO conc (ppm)']
y = NewData['Sum New Cases']

#--------------------------------------------------------------
# Linear regression model
#--------------------------------------------------------------
# Stastical approach
x_lms = sm.add_constant(x)
linear_model_stat = sm.OLS(y, x_lms)
lms_results = linear_model_stat.fit()
p_values = lms_results.pvalues[1:]

# Machine learning approach (no p-values)
linear_model = LinearRegression()
linear_model.fit(x_lms, y)
y_linear_pred = linear_model.predict(x_lms)

# Calculate R2 and RMSE for linear regression model
linear_r2 = r2_score(y, y_linear_pred)
linear_rmse = np.sqrt(mean_squared_error(y, y_linear_pred))

print("Model Evaluation")
print("\nLinear Regression: RMSE=%.2f, R2=%.2f" % (linear_rmse, linear_r2))
for i, p_value in enumerate(p_values.index):
  print(f'P({p_value}): {p_values[i]}')

if any(p_values <= 0.05):
  print("There is a significant relationship between the predictor and the response\n")
else:
  print("There is no significant relationship between the predictor and the response\n")


# Plot the model
import plotly.graph_objects as go

xtt = NewData['CO conc (ppm)']#.dt.strftime('%Y-%m-%d')
scatter_actual = go.Scatter(x=xtt, y=y, mode='markers', name='Actual', marker=dict(color='blue', opacity=0.3))

line_regression = go.Scatter(x=xtt, y=y_linear_pred, mode='lines', name='LR', line=dict(color='red', dash='dash'))

data = [scatter_actual, line_regression]

layout = go.Layout(xaxis=dict(title='CO conc (ppm)'),
                   yaxis=dict(title='Sum New Cases'),
                   title='Linear Regression Model for COVID-19 New Cases & CO conc (ppm)',
                   showlegend=True,
                   height=600)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
# For NO2 conc (ppb) & Sum New Cases
## Perform regression modeling
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

import statsmodels.api as sm
import calendar

# Convert datetime to int
#x = data['Date'].values.astype(int) / 10**9  # Convert to seconds (UNIX epoch start)
#x = x.reshape(-1, 1)


x = NewData['NO2 conc (ppb)']
y = NewData['Sum New Cases']

#--------------------------------------------------------------
# Linear regression model
#--------------------------------------------------------------
# Stastical approach
x_lms = sm.add_constant(x)
linear_model_stat = sm.OLS(y, x_lms)
lms_results = linear_model_stat.fit()
p_values = lms_results.pvalues[1:]

# Machine learning approach (no p-values)
linear_model = LinearRegression()
linear_model.fit(x_lms, y)
y_linear_pred = linear_model.predict(x_lms)

# Calculate R2 and RMSE for linear regression model
linear_r2 = r2_score(y, y_linear_pred)
linear_rmse = np.sqrt(mean_squared_error(y, y_linear_pred))

print("Model Evaluation")
print("\nLinear Regression: RMSE=%.2f, R2=%.2f" % (linear_rmse, linear_r2))
for i, p_value in enumerate(p_values.index):
  print(f'P({p_value}): {p_values[i]}')

if any(p_values <= 0.05):
  print("There is a significant relationship between the predictor and the response\n")
else:
  print("There is no significant relationship between the predictor and the response\n")


# Plot the model
import plotly.graph_objects as go

xtt = NewData['NO2 conc (ppb)']#.dt.strftime('%Y-%m-%d')
scatter_actual = go.Scatter(x=xtt, y=y, mode='markers', name='Actual', marker=dict(color='blue', opacity=0.3))

line_regression = go.Scatter(x=xtt, y=y_linear_pred, mode='lines', name='LR', line=dict(color='red', dash='dash'))

data = [scatter_actual, line_regression]

layout = go.Layout(xaxis=dict(title='NO2 conc (ppb)'),
                   yaxis=dict(title='Sum New Cases'),
                   title='Linear Regression Model for COVID-19 New Cases & NO2 conc (ppb)',
                   showlegend=True,
                   height=600)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
# For O3 conc (ppm) & Sum New Cases
## Perform regression modeling
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

import statsmodels.api as sm
import calendar

# Convert datetime to int
#x = data['Date'].values.astype(int) / 10**9  # Convert to seconds (UNIX epoch start)
#x = x.reshape(-1, 1)


x = NewData['O3 conc (ppm)']
y = NewData['Sum New Cases']

#--------------------------------------------------------------
# Linear regression model
#--------------------------------------------------------------
# Stastical approach
x_lms = sm.add_constant(x)
linear_model_stat = sm.OLS(y, x_lms)
lms_results = linear_model_stat.fit()
p_values = lms_results.pvalues[1:]

# Machine learning approach (no p-values)
linear_model = LinearRegression()
linear_model.fit(x_lms, y)
y_linear_pred = linear_model.predict(x_lms)

# Calculate R2 and RMSE for linear regression model
linear_r2 = r2_score(y, y_linear_pred)
linear_rmse = np.sqrt(mean_squared_error(y, y_linear_pred))

print("Model Evaluation")
print("\nLinear Regression: RMSE=%.2f, R2=%.2f" % (linear_rmse, linear_r2))
for i, p_value in enumerate(p_values.index):
  print(f'P({p_value}): {p_values[i]}')

if any(p_values <= 0.05):
  print("There is a significant relationship between the predictor and the response\n")
else:
  print("There is no significant relationship between the predictor and the response\n")


# Plot the model
import plotly.graph_objects as go

xtt = NewData['O3 conc (ppm)']#.dt.strftime('%Y-%m-%d')
scatter_actual = go.Scatter(x=xtt, y=y, mode='markers', name='Actual', marker=dict(color='blue', opacity=0.3))

line_regression = go.Scatter(x=xtt, y=y_linear_pred, mode='lines', name='LR', line=dict(color='red', dash='dash'))

data = [scatter_actual, line_regression]

layout = go.Layout(xaxis=dict(title='O3 conc (ppm)'),
                   yaxis=dict(title='Sum New Cases'),
                   title='Linear Regression Model for COVID-19 New Cases & O3 conc (ppm)',
                   showlegend=True,
                   height=600)

fig = go.Figure(data=data, layout=layout)
fig.show()